In [5]:
from sympy import symbols, sin, cos, pi, pprint, simplify, atan2, sqrt, atan
from sympy.matrices import Matrix, Transpose
from IPython.display import display
import ipywidgets as widgets
from functools import reduce
import numpy as np
from importlib import reload

In [11]:
import kuka_arm.scripts.inverse_kinematics as ik
reload(ik)

test_cases = {1:[[[2.16135,-1.42635,1.55109],
                  [0.708611,0.186356,-0.157931,0.661967]],
                  [1.89451,-1.44302,1.69366],
                  [-0.65,0.45,-0.36,0.95,0.79,0.49]],
              2:[[[-0.56754,0.93663,3.0038],
                  [0.62073, 0.48318,0.38759,0.480629]],
                  [-0.638,0.64198,2.9988],
                  [-0.79,-0.11,-2.33,1.94,1.14,-3.68]],
              3:[[[-1.3863,0.02074,0.90986],
                  [0.01735,-0.2179,0.9025,0.371016]],
                  [-1.1669,-0.17989,0.85137],
                  [-2.99,-0.12,0.94,4.06,1.29,-4.12]],
              4:[],
              5:[]}

ik.SolveIK(*test_cases[3][0])

wc_pos
⎡-1.16693608404 ⎤
⎢               ⎥
⎢-0.17988319325 ⎥
⎢               ⎥
⎣0.8513793265916⎦
computed wc_pos
⎡-1.16703264412961 ⎤
⎢                  ⎥
⎢-0.179898077987474⎥
⎢                  ⎥
⎣0.850575517883546 ⎦
R0_EE
⎡-0.72397332    -0.67724501   -0.1303720228  -1.3863⎤
⎢                                                   ⎥
⎢ 0.66212275   -0.629614545   -0.4061837552  0.02074⎥
⎢                                                   ⎥
⎢0.1930055228  -0.3804352448   0.904437135   0.90986⎥
⎢                                                   ⎥
⎣     0              0              0           1   ⎦
computed R0_EE
⎡-0.724039116899804  -0.677330782557284  -0.130347106601142  -1.20652781742976
⎢                                                                             
⎢0.662196784866248   -0.629696246463963  -0.406174907277641  -0.30296907489259
⎢                                                                             
⎢0.193035684037319   -0.380401956080057  0.904450980705262    1.124

Matrix([
[ -2.98864652138354],
[-0.115711289482777],
[ 0.943375365959017],
[  2.67223386216161],
[  2.13762992819175],
[ -1.24029704400537]])

In [ ]:
def DHTransform(alpha, a, q, d):
  return Matrix([
    [cos(q),            -sin(q),           0,           a],
    [sin(q)*cos(alpha), cos(q)*cos(alpha), -sin(alpha), -sin(alpha)*d],
    [sin(q)*sin(alpha), cos(q)*sin(alpha), cos(alpha),  cos(alpha)*d ],
    [0,                 0,                 0,           1            ]])

q1, q2, q3, q4, q5, q6 = symbols('q1:7')
alpha = [0, -pi/2, 0, -pi/2, pi/2, -pi/2, 0] #symbols('alpha0:7')
a = symbols('a0:7')
d1, d2, d3, d4, d5, d6, d_ee = symbols('d1:7,d_ee')

constants = dict(
  list(zip(a, [0, 0.35, 1.25, -0.054, 0, 0, 0])) +
  list(zip([d1, d2, d3, d4, d5, d6, d_ee], [0.75, 0, 0, 1.5, 0, 0, 0.303])))

T0_1 = DHTransform(alpha[0], a[0], q1, d1)
T1_2 = DHTransform(alpha[1], a[1], q2 - pi/2, d2)
T2_3 = DHTransform(alpha[2], a[2], q3, d3)
T3_4 = DHTransform(alpha[3], a[3], q4, d4)
T4_5 = DHTransform(alpha[4], a[4], q5, d5)
T5_6 = DHTransform(alpha[5], a[5], q6, d6)
T6_EE = DHTransform(alpha[6], a[6], 0, d_ee)

# def e(q11, q22, q33, q44, q55, q66):
#   subs = {q1: q11, q2: q22, q3: q33, q4: q44, q5: q55, q6: q66}
#   subs.update(constants)
#   return pprint(mat.evalf(subs=subs, prec = 5))

# spec = (-3.14,3.14,0.01)
# widgets.interact(e, q11=spec, q22=spec, q33=spec, q44=spec, q55=spec, q66=spec)

In [ ]:
mat = reduce(lambda acc, M: simplify(acc * M), [T0_1,T1_2,T2_3,T3_4,T4_5,T5_6,T6_EE])
subs = {q1: 2.86, q2: 0, q3: 0, q4:0 , q5: 0, q6: 0}
subs.update(constants)
mat.evalf(subs=subs, prec = 5)

In [2]:
def DHTransform(alpha, a, q, d):
  return Matrix([
    [cos(q),            -sin(q),           0,           a],
    [sin(q)*cos(alpha), cos(q)*cos(alpha), -sin(alpha), -sin(alpha)*d],
    [sin(q)*sin(alpha), cos(q)*sin(alpha), cos(alpha),  cos(alpha)*d ],
    [0,                 0,                 0,           1            ]])

q1, q2, q3, q4, q5, q6 = symbols('q1:7')
alpha = [0, -pi/2, 0, -pi/2, pi/2, -pi/2, 0]
a = [0, 0.35, 1.25, -0.054, 0, 0, 0]
d1, d2, d3, d4, d5, d6, d_ee = [0.75, 0, 0, 1.5, 0, 0, 0.303]

T0_1 = DHTransform(alpha[0], a[0], q1, d1)
T1_2 = DHTransform(alpha[1], a[1], q2 - pi/2, d2)
T2_3 = DHTransform(alpha[2], a[2], q3, d3)
T3_4 = DHTransform(alpha[3], a[3], q4, d4)
T4_5 = DHTransform(alpha[4], a[4], q5, d5)
T5_6 = DHTransform(alpha[5], a[5], q6, d6)
T6_EE = DHTransform(alpha[6], a[6], 0, d_ee)

In [86]:
def e(q11, q22, q33, q44, q55, q66, joints):
  mat = reduce(lambda acc, M: acc * M, [T1_2,T2_3,T3_4,T4_5,T5_6,T6_EE][:(joints-1)], T0_1)
  return pprint(mat.evalf(subs={q1: q11, q2: q22, q3: q33, q4: q44, q5: q55, q6: q66}, prec = 5))

spec = (-3.14,3.14,0.01)
widgets.interact(e, q11=spec, q22=spec, q33=spec, q44=spec, q55=spec, q66=spec, joints=(2,7))

⎡-1.1796e-16  -5.8981e-17      1.0         1.85    ⎤
⎢                                                  ⎥
⎢5.8981e-17      -1.0      -5.8981e-17  -1.0911e-16⎥
⎢                                                  ⎥
⎢    1.0      5.8981e-17   1.1796e-16      1.946   ⎥
⎢                                                  ⎥
⎣     0            0            0           1.0    ⎦


<function __main__.e>

In [3]:
def rot_x(q):
    R_x = Matrix([[ 1,              0,        0],
                  [ 0,         cos(q),  -sin(q)],
                  [ 0,         sin(q),  cos(q)]])
    
    return R_x
    
def rot_y(q):              
    R_y = Matrix([[ cos(q),        0,  sin(q)],
                  [      0,        1,       0],
                  [-sin(q),        0, cos(q)]])
    
    return R_y

def rot_z(q):    
    R_z = Matrix([[ cos(q),  -sin(q),       0],
                  [ sin(q),   cos(q),       0],
                  [      0,        0,       1]])
    
    return R_z

def quat2mat(quat):
    ''' Symbolic conversion from quaternion to rotation matrix
    For a unit quaternion
    
    From: http://en.wikipedia.org/wiki/Rotation_matrix#Quaternion
    Originally from: https://github.com/matthew-brett/transforms3d
    '''
    x, y, z, w = quat
    return Matrix([
            [1 - 2*y*y-2*z*z, 2*x*y - 2*z*w, 2*x*z+2*y*w],
            [2*x*y+2*z*w, 1-2*x*x-2*z*z, 2*y*z-2*x*w],
            [2*x*z-2*y*w, 2*y*z+2*x*w, 1-2*x*x-2*y*y]])

def joinz(R, T):
  return R.row_join(T).col_join(Matrix([[0,0,0,1]]))


In [88]:
s = '''
0.3631; -0.012039; 3.4913
-0.69656; -0.098579; -0.38459; 0.59765
'''.strip().split('\n')
ee_pos_vals = list(map(float, s[0].split('; ')))
quaternion_vals = list(map(float, s[1].split('; ')))
ee_pos_vals, quaternion_vals

([0.3631, -0.012039, 3.4913], [-0.69656, -0.098579, -0.38459, 0.59765])

In [4]:
test_cases = {1:[[[2.16135,-1.42635,1.55109],
                  [0.708611,0.186356,-0.157931,0.661967]],
                  [1.89451,-1.44302,1.69366],
                  [-0.65,0.45,-0.36,0.95,0.79,0.49]],
              2:[[[-0.56754,0.93663,3.0038],
                  [0.62073, 0.48318,0.38759,0.480629]],
                  [-0.638,0.64198,2.9988],
                  [-0.79,-0.11,-2.33,1.94,1.14,-3.68]],
              3:[[[-1.3863,0.02074,0.90986],
                  [0.01735,-0.2179,0.9025,0.371016]],
                  [-1.1669,-0.17989,0.85137],
                  [-2.99,-0.12,0.94,4.06,1.29,-4.12]],
              4:[],
              5:[]}
ee_pos_vals, quaternion_vals = test_cases[2][0]

In [5]:
R_gripper = rot_z(pi) * rot_y(-pi/2)

# ee_pos = symbols('ee_x,ee_y,ee_z')
quaternion = symbols('qx,qy,qz,qw')
constants = {}
# T = joinz(quat2mat(quaternion), Matrix(ee_pos))

ee_pos = Matrix(ee_pos_vals)
T = joinz(quat2mat(quaternion_vals), ee_pos)
# Construct homogenous transformaiton matrix based on the data.

R0_EE = T[0:3,0:3] * R_gripper
v_EE = T[0:3,3]
wc_pos = simplify(v_EE - R0_EE*Matrix([0,0,d_ee]))
wc_pos

Matrix([
[ -0.638024514177],
[0.64198594280094],
[2.99873479718912]])

In [6]:
xc, yc, zc = wc_pos
qq1 = atan2(yc, xc)

# q3
dd = sqrt(0.054**2 + 1.5**2)
dz = d1
dx = a[1]
s = zc - dz
r = sqrt(xc**2 + yc**2) - dx
cosbet = (r**2 + s**2 - a[2]**2 - dd**2)/(2*a[2]*dd)
sinbet = sqrt(1 - cosbet**2)

qq3 = -atan2(cosbet, sinbet) + atan2(-0.054, 1.5) 

# q2
qq2 = -(atan2(s, r) - atan2(a[2] + d4*cosbet, d4*sinbet))

# print(qq1, qq2.evalf(), qq3.evalf())

In [7]:
#R0_3 = simplify((T0_1 * T1_2 * T2_3)[0:3,0:3]).subs([(q1, qq1), (q2, qq2), (q3, qq3)])
T0_3 = T0_1*T1_2*T2_3
R0_3 = T0_3[0:3, 0:3].evalf(subs = {q1: qq1, q2: qq2, q3: qq3})
R3_EE_given = Transpose(R0_3)*R0_EE*R_gripper

sinq5 = sqrt(R3_EE_given[1,1]**2 + R3_EE_given[1, 2]**2)
cosq5 = R3_EE_given[1,0]
qq5 = atan2(sinq5, cosq5)

cosq4 = R3_EE_given[0,0]*-1
sinq4 = R3_EE_given[2,0]
qq4 = atan2(sinq4, cosq4)

cosq6 = R3_EE_given[1, 2] 
sinq6 = R3_EE_given[1, 1]
qq6 = atan2(sinq6, cosq6)

# print(qq4,qq5, qq6.evalf())

In [8]:
data = dict(
  list(zip(ee_pos, ee_pos_vals)) +
  list(zip(quaternion, quaternion_vals))
)
data.update(constants)

Matrix([qq1,qq2,qq3,qq4,qq5,qq6]).evalf(subs=data)
# (T0_1 * T1_2 * T2_3).subs([(q1, qq1), (q2, qq2), (q3, qq3)]).evalf(subs=data)

Matrix([
[  2.35309965924402],
[-0.389073094492763],
[-0.461667003145883],
[ -1.14646866021951],
[  1.20331812673337],
[  2.45544980510248]])

In [9]:
data={}
# pprint(t03)
# pprint(simplify(t03 * T3_4_sym.subs(constants)))
pprint((T0_1 * T1_2 * T2_3 * T3_4).subs([(q1, qq1), (q2, qq2), (q3, qq3), (q4, 0)]))
pprint(T)
pprint((T0_1 * T1_2 * T2_3 * T3_4 * T4_5 * T5_6 * T6_EE).subs([(q1, qq1), (q2, qq2), (q3, qq3), (q4, qq4), (q5, qq5), (q6, qq6)]))

⎡0.529933015681092   0.709291767319263  -0.464839959237866  -0.638351474359571
⎢                                                                             
⎢-0.533223315301942  0.704915022405621  0.467726102762385   0.642314933045675 
⎢                                                                             
⎢0.659426944330871           0          0.751768651308666    2.99862024849797 
⎢                                                                             
⎣        0                   0                  0                   1         

⎤
⎥
⎥
⎥
⎥
⎥
⎦
⎡ 0.232622159      0.22727465458     0.94563812184   -0.56754⎤
⎢                                                            ⎥
⎢0.97242263102  -0.0710634820000001  -0.22213020594  0.93663 ⎥
⎢                                                            ⎥
⎢0.01671684096     0.97123315074     -0.2375372906    3.0038 ⎥
⎢                                                            ⎥
⎣      0                 0                 0          

In [ ]:
T3_EE = simplify(T3_4*T4_5*T5_6*T6_EE)
R3_EE = T3_EE[0:3, 0:3]
pprint(simplify(R3_EE*R_gripper))
#pprint(simplify(sqrt(R3_EE[0,2]**2 + R3_EE[2, 2]**2)))

In [ ]:
atan2(yc, xc).evalf(subs=data)

In [ ]:
qs = Matrix([qq1,qq2,qq3,qq4,qq5,qq6]).evalf(subs=data)
dd = dict(zip([q1,q2,q3,q4,q5,q6], qs))
dd.update(constants)
mat.evalf(subs=dd)